In [38]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import t
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

def return_calculate(price,method):
    names=price.columns
    num=len(names)
    names_2=names[names!='Date']

    if num==len(names_2):
        print('No Date column in the price data')
    else:
        num=num-1

    pos=np.where(names!='Date')[0]
    row_num=price.shape[0]
    out=np.zeros([row_num-1,num])
    for i in range(num):
        temp=np.array(price.iloc[:,pos[i]])

        if method.upper()=='DISCRETE':
            out[:,i]=temp[1:]/temp[0:-1]-1
        elif method.upper()=='LOG':
            out[:,i]=np.log(temp[1:])-np.log(temp[0:-1])
        else:
            print('method must in Log or Discrete')

    out=pd.DataFrame(out)
    out.index=price.index[1:]
    out.columns=price.columns[pos]

    return out

price=pd.read_csv('DailyPrices.csv')
rt=return_calculate(price,method='Discrete')

pos_2=np.where(rt.columns=='META')[0][0]
rt_META=rt.iloc[:,pos_2]
rt_META=rt_META-np.mean(rt_META)

#1 Normal distribution
sigma=np.std(rt_META)
VAR_normal=-norm.ppf(0.05,0,sigma)

#2 EWV lambda = 0.94
lamda=0.94
row_num=price.shape[0]
wt=(1-lamda)*lamda**np.arange(row_num-2,-1,-1)
wt=wt/np.sum(wt)
sigma_exp=np.sqrt(np.sum(wt*rt_META**2))
VAR_normal_exp=-norm.ppf(0.05,0,sigma_exp)

#3 MLE with T distribution
sd=np.std(rt_META)
def likelihood(param):
    return -t.logpdf(np.array(rt_META/sd),param).sum()
rs = minimize(likelihood,5)
degree=rs.x[0]
VAR_t=-t.ppf(0.05,degree)*sd

#4 AR(1)
model=LinearRegression()
x=np.array(rt_META[0:-1])
y=np.array(rt_META[1:])
model.fit(x.reshape(-1,1),y.reshape(-1,1))
sd=np.std(rt_META)
VAR_AR=-(model.predict(y[-1].reshape(-1,1))+norm.ppf(0.05,0,sd))
VAR_AR=VAR_AR[0][0]

#5 Historical
sample=np.random.choice(np.array(rt_META),size=10000,replace=True)
VAR_HS=-np.percentile(sample,5)

#output
VAR=np.array([VAR_normal,VAR_normal_exp,VAR_t,VAR_AR,VAR_HS])
VAR=pd.DataFrame(VAR)
VAR.index=['Normal distribution','EWV lambda = 0.94 ','MLE with T distribution','AR(1)','Historical']
VAR.columns=['VAR']
print(VAR)

                              VAR
Normal distribution      0.065469
EWV lambda = 0.94        0.091385
MLE with T distribution  0.073142
AR(1)                    0.065749
Historical               0.055907


In [39]:
pt=pd.read_csv('portfolio.csv')
ls=['A','B','C']
stock_names=rt.columns
VAR_portfolio=np.zeros(3)
lamda = 0.94
wt = (1 - lamda) * lamda ** np.arange(row_num - 2, -1, -1)
wt = wt / np.sum(wt)
P0=100
portfolio_val = pd.DataFrame(np.zeros([price.shape[0], 3]))
for i in range(3):
    pos_temp=np.where(pt.iloc[:,0]==ls[i])[0]
    stock_num=len(pos_temp)
    for j in range(stock_num):
        pos_temp_2=np.where(pt.iloc[pos_temp[j],1]==stock_names)[0][0]+1
        portfolio_val.iloc[:,i]+=np.array(price.iloc[:,pos_temp_2])*pt.iloc[pos_temp[j],2]

portfolio_rt=return_calculate(portfolio_val, 'Discrete')
for i in range(3):
    sigma_exp = np.sqrt(np.sum(wt * (portfolio_rt.iloc[:,i]-np.mean(portfolio_rt.iloc[:,i])) ** 2))
    VAR_portfolio[i] = -norm.ppf(0.05, 0, sigma_exp)*portfolio_val.iloc[-1,i]

portfolio_val_total=np.sum(portfolio_val,1)
portfolio_rt_total=return_calculate(pd.DataFrame(portfolio_val_total), 'Discrete')
sigma_exp = np.sqrt(np.sum(wt * (np.array(portfolio_rt_total)-np.mean(np.array(portfolio_rt_total))) ** 2))
VAR_total = -norm.ppf(0.05, 0, sigma_exp)*portfolio_val_total.iloc[-1]

VAR_rs=np.append(VAR_portfolio,VAR_total)
VAR_rs=pd.DataFrame(VAR_rs)
VAR_rs.index=['A','B','C','total']
VAR_rs.columns=['VAR']
print(VAR_rs)

print("A+B+C =", sum(VAR_portfolio),"and total is:",VAR_total)

No Date column in the price data
No Date column in the price data
                 VAR
A        5603.790196
B        4371.694340
C        3748.704341
total  311238.346331
A+B+C = 13724.188876412196 and total is: 311238.34633069794


In [40]:
model=LinearRegression()
for i in range(3):
    rt_temp=np.array(portfolio_rt.iloc[:,i])
    x = np.array(rt_temp[0:-1])
    y = np.array(rt_temp[1:])
    model.fit(x.reshape(-1, 1), y.reshape(-1, 1))
    sd=np.std(portfolio_rt.iloc[:,i])
    temp = -(model.predict(y[-1].reshape(-1, 1)) + norm.ppf(0.05, 0, sd))
    VAR_portfolio[i] = temp[0][0]*portfolio_val.iloc[-1,i]

rt_temp=np.array(portfolio_rt_total.iloc[:,0])
x = np.array(rt_temp[0:-1])
y = np.array(rt_temp[1:])
model.fit(x.reshape(-1, 1), y.reshape(-1, 1))
sd=np.std(np.array(portfolio_rt_total))
temp = -(model.predict(y[-1].reshape(-1, 1)) + norm.ppf(0.05, 0, sd))
VAR_total = temp[0][0]*portfolio_val_total.iloc[-1]

VAR_rs=np.append(VAR_portfolio,VAR_total)
VAR_rs=pd.DataFrame(VAR_rs)
VAR_rs.index=['A','B','C','total']
VAR_rs.columns=['VAR']
print(VAR_rs)
print("A+B+C =", sum(VAR_portfolio),"and total is:",VAR_total)

                VAR
A       8004.338436
B       6560.836056
C       5647.895278
total  19889.905815
A+B+C = 20213.069770475195 and total is: 19889.905815313316
